In [1]:
#rdkit imports
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import Descriptors
from rdkit.Chem import rdFMCS
from rdkit.Chem.rdmolops import RDKFingerprint
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Avalon.pyAvalonTools import GetAvalonFP

#housekeeping imports
import pandas as pd
import matplotlib
import numpy as np
import scipy as sp

Maximum common substructure (not currently in use)

In [2]:
# finds maximum common substructure among list of molecule objects
def get_mcs(input_df):      # gets a df containing SMILES strings of products
    
    mol_list = list(input_df['SMILES']) # converts product column to list

    #for i in range(len(mol_list)): # converts SMILES strings to molecule objects
        #if mol_list[i] = 'none'
        #mol_list[i] = Chem.MolFromSmiles(mol_list[i])

    #maximum_common_substructure = rdFMCS.FindMCS(mol_list)
    
    return mol_list #maximum_common_substructure

#Draw.MolToImage(maximum_common_substructure, size=(300, 300), kekulize=False, wedgeBonds=True)

Data cleaning

input_df = pd.read_csv('~/OneDrive/Documents/Python/CLASSES/DIRECT/metamoles/datasets/playground_df_cleaned_kegg_with_smiles.csv')

In [3]:
loaded = pd.read_csv('../../../big-datasets/master_df_featurized.csv')

In [16]:
loaded.tail()

,enzyme,product,reacts,PubChemID,SMILES,n_C,Mol,Fingerprint,n_H,n_O,n_N,n_P,n_S,n_X,n_DoU,MW
17006,EC-2.7.7.5,CPD-9650,0.0,25246060,CC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=...,60,<rdkit.Chem.rdchem.Mol object at 0x00000253122...,"[0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, ...",97,7,0,2,0,0,13,992.377
17007,EC-1.1.1.286,PENTACHLOROPHENOL,0.0,4269236,C1(=C(C(=C(C(=C1Cl)Cl)Cl)Cl)Cl)[O-],6,<rdkit.Chem.rdchem.Mol object at 0x00000253122...,"[1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, ...",0,1,0,0,0,5,4,265.330
17008,EC-1.14.11.9,CPD-288,0.0,176951,C1=CC(C(C=C1)O)O,6,<rdkit.Chem.rdchem.Mol object at 0x00000253122...,"[0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, ...",8,2,0,0,0,0,3,112.128
17009,EC-1.9.3.1,2-HEXAPRENYL-3-METHYL-6-METHOXY-14-BENZ,0.0,50986193,CC1=C(C(=C(C=C1O)OC)O)CC=C(C)CCC=C(C)CCC=C(C)C...,53,<rdkit.Chem.rdchem.Mol object at 0x00000253122...,"[1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, ...",82,3,0,0,0,0,13,767.236
17010,EC-2.7.1.71,CHORISMATE,0.0,5460312,C=C(C(=O)[O-])OC1C=C(C=CC1O)C(=O)[O-],10,<rdkit.Chem.rdchem.Mol object at 0x00000253122...,"[0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, ...",8,6,0,0,0,0,7,224.168


In [15]:
pos = loaded[loaded['reacts'] == 1.0]

In [17]:
neg = loaded[loaded['reacts'] == 0.0]

In [51]:
def input_data(input_df): #cleans input df and returns neccessary elements
    '''DocString'''
    
    for index, row in input_df.iterrows():
        
        smiles = row['SMILES'] 
        if len(smiles) <= 2:
            
            input_df.drop(index, inplace=True)            
        
    return input_df

In [5]:
#input_df = input_data(input_df)

In [6]:
def fingerprint_products(input_df): #fingerprints all products in a given df
    '''DocString'''
    
    mol_list = []
    fp_list = []
    
    for index, row in input_df.iterrows():
        mol_list.append(Chem.rdmolfiles.MolFromSmiles(row['SMILES'])) #get mols from SMILES and add mols to list
        fp_list.append(FingerprintMols.FingerprintMol(Chem.rdmolfiles.MolFromSmiles(row['SMILES']))) #get fingerprints from mols and and fingerprints to list
        
    input_df.insert(6, column='Mol', value=mol_list)
    input_df.insert(7, column='Fingerprint', value= fp_list)
            
    return input_df

In [57]:
#input_df2 = fingerprint_products(input_data(input_df)) #build df containing new Mol and Fingerprint columns

split by enzyme

In [23]:
def split_by_enzyme(input_df):
    '''DocString'''
    
    unique_enzymes = set(input_df['enzyme'].unique())
    
    enzyme_df_list = []
    
    for entry in unique_enzymes: #for each unique enzyme in the input dataframe...
        
        enzyme_df = pd.DataFrame(columns=input_df.columns) #...initialize a new dataframe with the same columns as the input dataframe...
        
        for index, row in input_df.iterrows(): #...iterate through the input dataframe...
            
            if row['enzyme'] == entry: #... and add product rows that correspond to the unique enzyme entry...
                enzyme_df.loc[index] = row
                
        enzyme_df_list.append(enzyme_df) #...then add the completed dataframe of unique enzyme products to a list
           
    return enzyme_df_list #return list of dataframes

In [42]:
split_by_enzyme(test)

[          enzyme         product  reacts PubChemID  \
 82  EC-1.14.19.3    PALMITYL-COA     1.0  45266605   
 83  EC-1.14.19.3          CPD-18     1.0  25245440   
 84  EC-1.14.19.3  LINOLENOYL-COA     1.0  49859601   
 85  EC-1.14.19.3       CPD-17331     1.0  71581213   
 
                                                SMILES n_C n_H n_O n_N n_P n_S  \
 82  CCCCCCCCCCCCCCCC(=O)SCCNC(=O)CCNC(=O)C(C(C)(C)...  37  62  17   7   3   1   
 83  CCCCCC=CCC=CCCCCCCCC(=O)SCCNC(=O)CCNC(=O)C(C(C...  39  62  17   7   3   1   
 84  CCC=CCC=CCC=CCCCCCCCC(=O)SCCNC(=O)CCNC(=O)C(C(...  39  60  17   7   3   1   
 85  CCC=CCC=CCC=CCC=CCC=CCCCCCCCC(=O)SCCNC(=O)CCNC...  45  68  17   7   3   1   
 
    n_X n_DoU        MW  
 82   0    12  1001.924  
 83   0    14  1025.946  
 84   0    15  1023.930  
 85   0    17  1104.060  ,           enzyme    product  reacts PubChemID  \
 88  EC-1.1.1.384  CPD-17531     1.0  86583395   
 
                                  SMILES n_C n_H n_O n_N n_P n_S n_X n_DoU  \
 

In [24]:
def main(input_df):
    '''DocString'''
        
    input_df = fingerprint_products(input_data(input_df))    #expand input df: generate mols from SMILES then generate fingerprints from mols, adding columns for each
    
    enzyme_df_list = split_by_enzyme(input_df)    #split expanded df by rows, grouped by enzyme entry (1.1.1.110 etc), into a list of dataframes
    
    for enzyme_df in enzyme_df_list:    #loop through list of enzyme dataframes
        
        enzyme_df['Dist'] = '' #initialize distance column
        
        metric = sim_metric(enzyme_df) #get similarity matrix dataframe
        
        vals = metric.values #use np array of similarity matrix
        
        start_at = 1 #skip autocorrelation
        
        dist_list =[] #initialize list
        
        for i in range(len(vals)-1): #row of matrix except for last row
            
            for j in range(start_at, len(vals)): #col of matrix skipping first column
                
                dist_list.append(vals[i][j]) #add distance value to list
            
            start_at += 1 #start at higher index to skip redundancy
        
        avg_dist = sum(dist_list)/len(dist_list) #compute average distance
        
        for index, row in enzyme_df.iterrows():    #loop through enzyme dataframe 
            enzyme_df['Dist'].loc[index] = avg_dist #add averaged distance to each product row of enzyme dataframe
    
    master_df = pd.concat(enzyme_df_list) #concatenate enzyme dataframes into master_df
    
    return master_df

In [44]:
pos = pos.drop(columns=['Mol', 'Fingerprint'])

KeyError: "['Mol' 'Fingerprint'] not found in axis"

In [52]:
test = pos.iloc[:1,:].copy()

In [53]:
pos_df = main(test)

ZeroDivisionError: division by zero

similarity functions

In [9]:
def sim_i_j(row_i, row_j):
    """DocString"""
    return DataStructs.FingerprintSimilarity(row_i['Fingerprint'], row_j['Fingerprint'], metric=DataStructs.TanimotoSimilarity)

In [10]:
def sim_i_all(input_df, index_i, row_i, metric):
    """DocString"""
    for index_j, row_j in input_df.iterrows():
        if index_j < index_i: #skip redundant rows
            continue
        elif index_i == index_j: #autocorrelate rows
            metric.loc[index_i, index_j] = 1
        else:
            metric.loc[index_i, index_j] = sim_i_j(row_i, row_j) #fill matrix with calculated similarity at two positions at once 
            metric.loc[index_j, index_i] = metric.loc[index_i, index_j]
    return 

In [11]:
def sim_metric(input_df):
    """DocString"""
    metric = pd.DataFrame()
    for index_i, row_i in input_df.iterrows():
        sim_i_all(input_df, index_i, row_i, metric)
    return metric

In [43]:
sim_metric(test)

KeyError: 'Fingerprint'